In [ ]:
!pip install nlpaug
!pip install googletrans==4.0.0-rc1

In [ ]:
import nlpaug.augmenter.char as nac
from googletrans import Translator
import string
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from nlpaug.util import Action

In [ ]:
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translator)
    return cleaned_text

def paraphrase_thai(text):
    # Translate to English
    translator = Translator()
    english_text = translator.translate(text, dest='en')
    en_text = remove_punctuation(english_text.text)

    # Paraphrase in English (you can use other libraries or models for this step)
    paraphrased_englishs = paraphrase_function(en_text)

    # Translate back to Thai
    th_texts =[]

    for paraphrased_english in paraphrased_englishs:
      paraphrased_thai = translator.translate(paraphrased_english, dest='th')
      th_text = remove_punctuation(paraphrased_thai.text)
      th_texts.append(th_text)

    return th_texts

def paraphrase_function(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to('cuda:0')

    outputs = model.generate(
        input_ids, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

def random_insert(text, aug_char_p=0.05):
    thai_letters = [chr(code) for code in range(0xE01, 0xE3A)]  # U+0E01 to U+0E39
    aug_insert = nac.RandomCharAug(action='insert', candidates=thai_letters, aug_char_p=aug_char_p)
    insert_augmented = aug_insert.augment(text)
    insert_augment_noweirdspace = insert_augmented[0].replace('\xa0', '')
    return insert_augment_noweirdspace.replace(' ', '')

def random_deletion(text, aug_char_p=0.05):
    aug_delete = nac.RandomCharAug(action='delete', aug_char_p=aug_char_p)
    delete_augmented = aug_delete.augment(text)
    delete_augment_noweirdspace = delete_augmented[0].replace('\xa0', '')
    return delete_augment_noweirdspace.replace(' ', '')

def random_swap(text, aug_char_p=0.05):
    aug_swap = nac.RandomCharAug(action='swap', aug_char_p=aug_char_p)
    swap_augmented = aug_swap.augment(text)
    swap_augment_noweirdspace = swap_augmented[0].replace('\xa0', ' ')
    return swap_augment_noweirdspace.replace(' ', '')

def back_translation(text, destination_language='en'):
    translator = Translator()
    translation = translator.translate(text, dest=destination_language)
    return translation.text

In [ ]:
# Sample Thai question
thai_question = "วิทยาศาสตร์ควอนตัมคืออะไร"

# Call the functions and store the augmented texts
insert_augmented = random_insert(thai_question)
delete_augmented = random_deletion(thai_question)
swap_augmented = random_swap(thai_question)
back_translation_result = back_translation(thai_question)
paraphrased_texts = paraphrase_thai(thai_question)

# Display the augmented texts
print(f"Random Insertion: {insert_augmented}")
print(f"Random Deletion: {delete_augmented}")
print(f"Random Swap: {swap_augmented}")
print(f"Back Translation: {back_translation_result}")
for paraphrased_text in paraphrased_texts:
  print(f"Paraphrased text: {paraphrased_text}")


Random Insertion: วิทยาศาบสตร์ควอนถตัมคือัอะไร
Random Deletion: วิทยาศาตร์ควอตัมคือะไร
Random Swap: วิทยาศาสรต์ควอตนัมคืออะไร
Back Translation: What is quantum science?
Paraphrased text: ธรรมชาติของวิทยาศาสตร์ควอนตัมคืออะไร
Paraphrased text: คุณจะอธิบายวิทยาศาสตร์ควอนตัมได้อย่างไร
Paraphrased text: คุณสามารถอธิบายแนวคิดของวิทยาศาสตร์ควอนตัมได้หรือไม่
Paraphrased text: คุณสามารถให้คำอธิบายเกี่ยวกับวิทยาศาสตร์ควอนตัมได้หรือไม่
Paraphrased text: คำว่า วิทยาศาสตร์ควอนตัม หมายถึงอะไร


In [ ]:
import pandas as pd

Question_Counter = 0
augment_counter = 0
# Read the CSV file
input_csv_path = 'qalist.csv'
output_csv_path = 'augmented_output_file.csv'

df = pd.read_csv(input_csv_path)

# Create a new DataFrame to store augmented data
augmented_df = pd.DataFrame(columns=df.columns)

# Augment each row in the original DataFrame and add as new rows in the augmented DataFrame
for index, row in df.iterrows():
    Question_Counter+=1
    print(f"Quesiton:{Question_Counter}")
    thai_question = row['question']
    index_class = row['class']

    aug_questions=[]

    aug_questions.append(random_insert(thai_question))
    aug_questions.append(random_insert(thai_question))
    aug_questions.append(random_deletion(thai_question))
    aug_questions.append(random_deletion(thai_question))
    aug_questions.append(random_swap(thai_question))
    aug_questions.append(random_swap(thai_question))
    aug_questions.extend(paraphrase_thai(thai_question))

    # Append augmented data as new rows in the augmented DataFrame
    for aug_question in aug_questions:
      augment_counter+=1
      print(f"Augment:{augment_counter}")
      aug_df = pd.DataFrame({
        'question': aug_questions,
        'class': index_class
      })

      # Concatenate the augmented DataFrame with the original DataFrame
      augmented_df = pd.concat([augmented_df, aug_df], ignore_index=True)

# Concatenate the original DataFrame with the augmented DataFrame
result_df = pd.concat([df, augmented_df], ignore_index=True)

# Save the result DataFrame to a new CSV file
result_df.to_csv(output_csv_path, index=False)

Quesiton:1
Augment:1
Augment:2
Augment:3
Augment:4
Augment:5
Augment:6
Augment:7
Augment:8
Augment:9
Augment:10
Augment:11
Quesiton:2


<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Augment:12
Augment:13
Augment:14
Augment:15
Augment:16
Augment:17
Augment:18
Augment:19
Augment:20
Augment:21
Augment:22
Quesiton:3


<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Augment:23
Augment:24
Augment:25
Augment:26
Augment:27
Augment:28
Augment:29
Augment:30
Augment:31
Augment:32
Augment:33
Quesiton:4


...

<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Augment:2212
Augment:2213
Augment:2214
Augment:2215
Augment:2216
Augment:2217
Augment:2218
Augment:2219
Augment:2220
Augment:2221
Augment:2222
Quesiton:203


<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Augment:2223
Augment:2224
Augment:2225
Augment:2226
Augment:2227
Augment:2228
Augment:2229
Augment:2230
Augment:2231
Augment:2232
Augment:2233


<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_df = augmented_df.append({
<ipython-input-15-e76c590d8437>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.